# -----------------------------ETL------------------------------

Análisis exploratorio de datos
Se realiza un análisis exploratorio de los datos, con la finalidad de encontrar patrones que permitan generar información para la toma de decisiones en relación a disminuir las víctimas fatales.
Para una lectura mas ordenada, los códigos de gráficos y otros datos se colocaron en funciones dentro de la librería functions que se adjunta en este repositorio.

In [2]:
import Funciones as func
import numpy as np
import pandas as pd
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings("ignore")
from datetime import datetime


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Traemos la pestaña Hechos del Excel de Homicidios
Se extraen los datos de la pestaña Hechos que se encuentra en el Excel de homicidios y se observan las primeras filas con .head()
Para que podamos leer archivos .xlsx con pandas, debemos tener instaladas las librerías:

* xlrd
* openpyxl
Más allá de que por defecto, pandas trae incorporada la función read_excel, si no tenemos esas dos librerías nos daría error al intentar ejecutar

In [71]:
df_hechos= pd.read_excel('Data_raw\homicidios.xlsx', sheet_name='HECHOS')

df_hechos.head(3)

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,...,2034.0,NaN,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO


Datos generales:

El conjunto de datos contiene 21 columnas y 696 registros

In [72]:
#Columnas del dataset

df_hechos.columns

Index(['ID', 'N_VICTIMAS', 'FECHA', 'AAAA', 'MM', 'DD', 'HORA', 'HH',
       'LUGAR_DEL_HECHO', 'TIPO_DE_CALLE', 'Calle', 'Altura', 'Cruce',
       'Dirección Normalizada', 'COMUNA', 'XY (CABA)', 'pos x', 'pos y',
       'PARTICIPANTES', 'VICTIMA', 'ACUSADO'],
      dtype='object')

In [73]:
#Registros totales del dataset

df_hechos.shape[0]

696

In [74]:
#Ya que algunas columnas estan escritas en mayuscula y otras en minúscula, se estandarizan los nombres.
# Se coloca la primera en mayúscula
df_hechos.columns = [x.capitalize() for x in df_hechos.columns]
# Se reemplazan los guiones por espacios
df_hechos.columns = df_hechos.columns.str.replace('_', ' ')
# Se renombran algunas columnas
df_hechos = df_hechos.rename(columns={'N victimas': 'Cantidad víctimas',
                                                      'Aaaa':'Año',
                                                      'Mm':'Mes',
                                                      'Dd':'Día',
                                                      'Hora':'Hora completa',
                                                      'Hh':'Hora',
                                                      'Victima': 'Víctima'})
df_hechos.columns

Index(['Id', 'Cantidad víctimas', 'Fecha', 'Año', 'Mes', 'Día',
       'Hora completa', 'Hora', 'Lugar del hecho', 'Tipo de calle', 'Calle',
       'Altura', 'Cruce', 'Dirección normalizada', 'Comuna', 'Xy (caba)',
       'Pos x', 'Pos y', 'Participantes', 'Víctima', 'Acusado'],
      dtype='object')

Datos nulos
En cuanto a los valores faltantes, hay 747 valores nulos, que representan el 5.1% de todos los datos y se ubican en las columnas 'Altura', 'Cruce', 'Calle' y 'Dirección normalizada'.

En la columna 'Altura' faltan el 81.5% de los datos, debido a que este dato se completa cuando el hecho ocurre en algún punto de la calle (altura de la calle) y la mayoría de los hechos registrados ocurrieron en esquinas. Por este motivo, se considera que no es un dato relevante para el análsis y se decide eliminarlo del conjunto. Adicionalmente, este dato se encuentra en la columna 'Lugar del hecho' y 'Dirección normalizada'.

In [75]:
print(df_hechos.isnull().sum())

Id                         0
Cantidad víctimas          0
Fecha                      0
Año                        0
Mes                        0
Día                        0
Hora completa              0
Hora                       0
Lugar del hecho            0
Tipo de calle              0
Calle                      1
Altura                   567
Cruce                    171
Dirección normalizada      8
Comuna                     0
Xy (caba)                  0
Pos x                      0
Pos y                      0
Participantes              0
Víctima                    0
Acusado                    0
dtype: int64


In [76]:
# Se elimina la columna
df_hechos =df_hechos.drop('Altura', axis=1)
df_hechos.columns

Index(['Id', 'Cantidad víctimas', 'Fecha', 'Año', 'Mes', 'Día',
       'Hora completa', 'Hora', 'Lugar del hecho', 'Tipo de calle', 'Calle',
       'Cruce', 'Dirección normalizada', 'Comuna', 'Xy (caba)', 'Pos x',
       'Pos y', 'Participantes', 'Víctima', 'Acusado'],
      dtype='object')

En cuanto a la colomna 'Cruce' es una variable categórica, de tipo string, que contiene 317 valones distintos y donde hay un 24,6% de valores faltantes. Al contrario de la variable anterior, esta variable no tiene dato cuando se trata de un hecho ocurrido en un cruce. Esta columna puede ser interesante para evaluar si los hechos ocurren en esquinas o en algún punto de la cuadra. A continuación, se observa la columna 'Cruce' en relación a 'Lugar del hecho' y 'Tipo de calle'.

In [77]:
# Se ven los valores nulos de cruce en relación con 'Lugar del hecho' y 'Tipo de calle'
df_hechos[df_hechos['Cruce'].isnull()][['Cruce', 'Lugar del hecho', 'Tipo de calle']][:15]

,Cruce,Lugar del hecho,Tipo de calle
2,NaN,AV ENTRE RIOS 2034,AVENIDA
9,NaN,AV ENTRE RIOS 1366,AVENIDA
14,NaN,SUIPACHA 156,CALLE
33,NaN,LIMA 1483,CALLE
35,NaN,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI KM....,AUTOPISTA
38,NaN,AUTOPISTA LUGONES PK 10000,AUTOPISTA
42,NaN,AV LUIS MARIA CAMPOS 30,AVENIDA
43,NaN,LIMA 1471,CALLE
50,NaN,MAGARIÑOS CERVANTES 3900,CALLE
59,NaN,AV LA PLATA 2384,AVENIDA


Se puede ver que los datos faltantes efectivamente son los que se refieren a un hecho en algún punto de la calle que no es una esquina o intersección de dos calles. Por lo tanto, se decide modificar esta columna en un valor 'si' para los casos que son hechos en cruces de calles y 'no' en el caso que no sea un cruce de calles.

In [78]:
# Se reemplazan los valores no nulos por "no" y los nulos por "si"
df_hechos['Cruce'] = np.where(df_hechos['Cruce'].notnull(), 'SI', 'NO')
# Se revisa la columna
df_hechos[['Cruce', 'Lugar del hecho', 'Tipo de calle']]

,Cruce,Lugar del hecho,Tipo de calle
0,SI,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA
1,SI,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ
2,NO,AV ENTRE RIOS 2034,AVENIDA
3,SI,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA
4,SI,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA
...,...,...,...
691,SI,AV. RIESTRA Y MOM,AVENIDA
692,SI,AU DELLEPIANE Y LACARRA,AUTOPISTA
693,SI,AV. GAONA Y TERRADA,AVENIDA
694,NO,AV. EVA PERON 4071,AVENIDA


Datos duplicados
No se observan filas duplicadas. Podemos ver que en la columna 'Id' tenemos el total de registros que son únicos

In [79]:
print(f"La columna 'Id' cuenta con {len(df_hechos['Id'].unique())} valores únicos.")

La columna 'Id' cuenta con 696 valores únicos.


Se comparan las columnas 'Lugar del hecho' y 'Dirección normalizada'

In [80]:
df_hechos[['Lugar del hecho', 'Dirección normalizada']]

,Lugar del hecho,Dirección normalizada
0,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,"PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G..."
1,AV GRAL PAZ Y AV DE LOS CORRALES,"PAZ, GRAL. AV. y DE LOS CORRALES AV."
2,AV ENTRE RIOS 2034,ENTRE RIOS AV. 2034
3,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,"LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL."
4,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,"SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES."
...,...,...
691,AV. RIESTRA Y MOM,RIESTRA AV. y MOM
692,AU DELLEPIANE Y LACARRA,"DELLEPIANE, LUIS, TTE. GRAL. y LACARRA AV."
693,AV. GAONA Y TERRADA,GAONA AV. y TERRADA
694,AV. EVA PERON 4071,"PERON, EVA AV. 4071"


Anteriormente vimos que la columna "Dirección normalizada" tiene 8 datos nulos, entonces vamos completarla con los datos de "Lugar del hecho".

In [83]:
#Competar los valores faltantes de la columna 'Dirección normalizada' con los de 'Lugar del hecho'
df_hechos['Dirección normalizada'] = df_hechos['Dirección normalizada'].fillna(df_hechos['Lugar del hecho'])

Se decide elimina la columna 'Lugar del hecho' y reemplazarla con 'Dirección normalizada' porque consideramos primero, que no perdemos ningun dato. Y, en segundo lugar, que en el caso de una futura búsqueda es últil poder utilizar el filtro alfabético. Mientras que si queremos buscar por tipo de calle tenemos la columna 'Tipo de calle'.

In [84]:
# Eliminar columna 'Lugar del hecho' y se reemplaza con 'Dirección normalizada' a la cual se le cambia el nombre 
df_hechos = df_hechos.drop(columns=['Lugar del hecho']).rename(columns={'Dirección normalizada': 'Lugar del hecho'})


Tipos de variables

En el informe, podemos observar varios tipos de variables en:

Fecha
Hora
Calle

In [85]:
#Vemos el tipo de dato en cada columna.
df_hechos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Id                 696 non-null    object        
 1   Cantidad víctimas  696 non-null    int64         
 2   Fecha              696 non-null    datetime64[ns]
 3   Año                696 non-null    int64         
 4   Mes                696 non-null    int64         
 5   Día                696 non-null    int64         
 6   Hora completa      696 non-null    object        
 7   Hora               696 non-null    object        
 8   Tipo de calle      696 non-null    object        
 9   Calle              695 non-null    object        
 10  Cruce              696 non-null    object        
 11  Lugar del hecho    696 non-null    object        
 12  Comuna             696 non-null    int64         
 13  Xy (caba)          696 non-null    object        
 14  Pos x     

columna 'Hora completa'

In [86]:
# Cantidad de valores por tipo de dato en la columna 'hora'
df_hechos['Hora completa'].apply(type).value_counts()

Hora completa
<class 'datetime.time'>        608
<class 'str'>                   85
<class 'datetime.datetime'>      3
Name: count, dtype: int64

In [87]:
# Funcion para convertir a time
def convertir_a_time(x):
    if isinstance(x, str):
        try:
            return datetime.strptime(x, "%H:%M:%S").time()
        except ValueError:
            return None
    elif isinstance(x, datetime):
        return x.time()
    return x

# Aplicar la función directamente a la columna 'Hora completa' de tu DataFrame
df_hechos['Hora completa'] = df_hechos['Hora completa'].apply(convertir_a_time)

# Verificar que se hayan modificado los valores
print(df_hechos['Hora completa'].apply(type).value_counts())

Hora completa
<class 'datetime.time'>    695
<class 'NoneType'>           1
Name: count, dtype: int64


In [88]:
# Buscar registro que no se modificó:
df_hechos[df_hechos['Hora completa'].isna()]

,Id,Cantidad víctimas,Fecha,Año,Mes,Día,Hora completa,Hora,Tipo de calle,Calle,Cruce,Lugar del hecho,Comuna,Xy (caba),Pos x,Pos y,Participantes,Víctima,Acusado
518,2019-0103,1,2019-12-18,2019,12,18,None,SD,GRAL PAZ,"PAZ, GRAL. AV.",SI,"PAZ, GRAL. AV. y GRIVEO",11,Point (94643.11254058 103831.57115061),-58.52169422,-34.59471640,MOTO-MOTO,MOTO,MOTO


In [89]:
#Calculamos la hora mas común
hora_moda = df_hechos['Hora completa'].mode().iloc[0]
print(f'La hora mas común es: {hora_moda}')

#Reemplazar el valor
df_hechos['Hora completa'].fillna(hora_moda, inplace=True)

#Verificar el tipo de dato para toda la columna
df_hechos['Hora completa'].apply(type).value_counts()

La hora mas común es: 09:00:00


Hora completa
<class 'datetime.time'>    696
Name: count, dtype: int64

Ahora modificamos el valor de la columna 'Hora', porque es de tipo 'str'

In [90]:
#verificamos el tipo de dato para toda la columna
df_hechos['Hora'].apply(type).value_counts()
#Agregamos el valor de la hora moda, en el dato faltante.
df_hechos['Hora'] = df_hechos['Hora'].apply(lambda x: int(hora_moda.hour) if x== 'SD' else x)

#Verificamos que se haya modificado el tipo de dato
df_hechos['Hora'].apply(type).value_counts()

Hora
<class 'int'>    696
Name: count, dtype: int64

In [91]:
# Visualizamos
df_hechos[df_hechos['Id']== '2019-0103']

,Id,Cantidad víctimas,Fecha,Año,Mes,Día,Hora completa,Hora,Tipo de calle,Calle,Cruce,Lugar del hecho,Comuna,Xy (caba),Pos x,Pos y,Participantes,Víctima,Acusado
518,2019-0103,1,2019-12-18,2019,12,18,09:00:00,9,GRAL PAZ,"PAZ, GRAL. AV.",SI,"PAZ, GRAL. AV. y GRIVEO",11,Point (94643.11254058 103831.57115061),-58.52169422,-34.59471640,MOTO-MOTO,MOTO,MOTO


la columna 'Calle' presenta dos tipos de datos. Por la información de la variable, es una columna de strings pero se presentan algunos float. Se revisan los registros con este último tipo de dato.    

In [92]:
# Se verifica el tipo de dato
print('Tipos de datos:')
print(df_hechos['Calle'].apply(type).value_counts())
# Se observa el registro con tipo de dato float
print('Registro con tipo float:')
df_hechos[df_hechos['Calle'].apply(lambda x: isinstance(x, float))]

Tipos de datos:
Calle
<class 'str'>      695
<class 'float'>      1
Name: count, dtype: int64
Registro con tipo float:


,Id,Cantidad víctimas,Fecha,Año,Mes,Día,Hora completa,Hora,Tipo de calle,Calle,Cruce,Lugar del hecho,Comuna,Xy (caba),Pos x,Pos y,Participantes,Víctima,Acusado
119,2016-0151,1,2016-11-18,2016,11,18,20:35:00,20,CALLE,NaN,NO,SD,0,Point (. .),.,.,PEATON-SD,PEATON,SD


Al no tener datos suficientes sobre la calle del hecho, se cambia el valor por SD(Sin Dato) para seguir con la idea original de los faltantes.

In [93]:
df_hechos['Calle'].fillna('SD', inplace=True)

#Verificamos que se haya realizado el cambio 
df_hechos[df_hechos['Id']== '2016-0151']

,Id,Cantidad víctimas,Fecha,Año,Mes,Día,Hora completa,Hora,Tipo de calle,Calle,Cruce,Lugar del hecho,Comuna,Xy (caba),Pos x,Pos y,Participantes,Víctima,Acusado
119,2016-0151,1,2016-11-18,2016,11,18,20:35:00,20,CALLE,SD,NO,SD,0,Point (. .),.,.,PEATON-SD,PEATON,SD


verificamos nuevamente los tipos de datos.

Categorías de Víctima y Acusado
En este apartado, vamos a verificar si las categorías que aparecen en el dataset, son las indicadas en el diccionario de datos.

In [94]:
#Traer los valores únicos de la columna 'Victima'
df_hechos['Víctima'].unique()

array(['MOTO', 'AUTO', 'PEATON', 'SD', 'CARGAS', 'BICICLETA', 'PASAJEROS',
       'MOVIL', 'OBJETO FIJO', 'PEATON_MOTO'], dtype=object)

En la columna Acusado están las categorías correctas, pero en la columna Víctima aparecen las categorías 'OBJETO FIJO' y 'PEATON_MOTO', las cuales no están indicadas en el diccionario de datos.

In [95]:
#Buscar los registros 
df_hechos[df_hechos['Víctima'].isin(['OBJETO FIJO', 'PEATON_MOTO'])]

,Id,Cantidad víctimas,Fecha,Año,Mes,Día,Hora completa,Hora,Tipo de calle,Calle,Cruce,Lugar del hecho,Comuna,Xy (caba),Pos x,Pos y,Participantes,Víctima,Acusado
230,2017-0108,2,2017-09-02,2017,9,2,04:53:08,4,GRAL PAZ,"PAZ, GRAL. AV.",SI,"PAZ, GRAL. AV. y MACHAIN",12,Point (97098.48468623 109019.96106626),-58.49491054,-34.54795581,AUTO-OBJETO FIJO,OBJETO FIJO,AUTO
583,2020-0063,2,2020-12-05,2020,12,5,07:10:00,7,CALLE,NUEVA YORK,SI,NUEVA YORK y ALTA GRACIA,11,Point (94080.62190808 102083.62453795),-58.52783814,-34.61047001,PEATON_MOTO-MOTO,PEATON_MOTO,MOTO


In [96]:
#Cambiar el dato de los dos registros por 'OTRO'
df_hechos['Víctima']= df_hechos['Víctima'].replace({'OBJETO FIJO':'OTRO', 'PEATON_MOTO':'OTRO'})

#Verificar que se haya realizado el cambio correctamente
df_hechos[df_hechos['Id'].isin(['2017-0108','2020-0063'])]

,Id,Cantidad víctimas,Fecha,Año,Mes,Día,Hora completa,Hora,Tipo de calle,Calle,Cruce,Lugar del hecho,Comuna,Xy (caba),Pos x,Pos y,Participantes,Víctima,Acusado
230,2017-0108,2,2017-09-02,2017,9,2,04:53:08,4,GRAL PAZ,"PAZ, GRAL. AV.",SI,"PAZ, GRAL. AV. y MACHAIN",12,Point (97098.48468623 109019.96106626),-58.49491054,-34.54795581,AUTO-OBJETO FIJO,OTRO,AUTO
583,2020-0063,2,2020-12-05,2020,12,5,07:10:00,7,CALLE,NUEVA YORK,SI,NUEVA YORK y ALTA GRACIA,11,Point (94080.62190808 102083.62453795),-58.52783814,-34.61047001,PEATON_MOTO-MOTO,OTRO,MOTO


In [97]:
#Traer los valores únicos de la columna 'Acusado'
df_hechos['Acusado'].unique()

array(['AUTO', 'PASAJEROS', 'SD', 'OBJETO FIJO', 'CARGAS', 'MOTO',
       'MULTIPLE', 'OTRO', 'BICICLETA', 'TREN'], dtype=object)

En la columna Acusado está la categoría PASAJEROS con la siguiente descripción: "En la columna Acusado están las categorías correctas, pero en la columna Víctima aparecen las categorías 'OBJETO FIJO' y 'PEATON_MOTO', las cuales no están indicadas en el diccionario de datos." Esto no se corresponde con la categoría de "ACUSADO" puesto que  en términos legales nunca un pasajero tiene la responsabilidad, sino el conductor.

In [98]:
#Buscar los registros 
df_hechos[df_hechos['Acusado'].isin(['PASAJEROS'])]

,Id,Cantidad víctimas,Fecha,Año,Mes,Día,Hora completa,Hora,Tipo de calle,Calle,Cruce,Lugar del hecho,Comuna,Xy (caba),Pos x,Pos y,Participantes,Víctima,Acusado
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,GRAL PAZ,"PAZ, GRAL. AV.",SI,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS
4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5,AVENIDA,SAN JUAN AV.,SI,"SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1,Point (106980.32827929 100752.16915795),-58.38718297,-34.62246630,MOTO-PASAJEROS,MOTO,PASAJEROS
13,2016-0019,1,2016-02-17,2016,2,17,09:30:00,9,CALLE,"DE LA TORRE, LISANDRO",SI,"DE LA TORRE, LISANDRO y GARCIA GRANDE DE ZEQUE...",9,Point (95727.37083199 97179.35560349),-58.50990884,-34.65468721,PEATON-PASAJEROS,PEATON,PASAJEROS
21,2016-0029,1,2016-03-04,2016,3,4,17:30:00,17,AVENIDA,CABILDO AV.,SI,CABILDO AV. y DEHEZA,12,Point (99037.31284333 109733.12041651),-58.47378664,-34.54153077,PEATON-PASAJEROS,PEATON,PASAJEROS
22,2016-0031,1,2016-03-08,2016,3,8,12:30:00,12,GRAL PAZ,"PAZ, GRAL. AV.",SI,"PAZ, GRAL. AV. y ROCA, CNEL. AV.",8,Point (99327.01838497 92326.91130631),-58.47064556,-34.69843756,MOTO-PASAJEROS,MOTO,PASAJEROS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
663,2021-0065,1,2021-08-06,2021,8,6,08:30:00,8,AVENIDA,SAN JUAN AV.,SI,SAN JUAN AV. y LA PLATA AV.,6,Point (103350.94520286 100251.53397151),-58.42675754,-34.62699730,PEATON-PASAJEROS,PEATON,PASAJEROS
664,2021-0066,1,2021-08-06,2021,8,6,20:00:00,20,AVENIDA,LOPE DE VEGA AV.,SI,"LOPE DE VEGA AV. y PAZ, GRAL. AV.",11,Point (93943.16495967 102026.75338495),-58.52933723,-34.61098186,PEATON-PASAJEROS,PEATON,PASAJEROS
676,2021-0078,1,2021-10-23,2021,10,23,06:35:00,6,AVENIDA,INDEPENDENCIA AV.,SI,INDEPENDENCIA AV. y LA PLATA AV.,6,Point (103255.81315650 100797.78656418),-58.42779702,-34.62207347,MOTO-PASAJEROS,MOTO,PASAJEROS
684,2021-0086,1,2021-11-25,2021,11,25,10:10:00,10,AVENIDA,RIVADAVIA AV.,SI,RIVADAVIA AV. y PUAN,6,Point (101343.57441195 100636.86479169),-58.44864864,-34.62352826,PEATON-PASAJEROS,PEATON,PASAJEROS


Consultado esto con la OMSV, se toma la decision de convertir esta categoria a 'TRANSPORTE DE PASAJEROS'.

In [99]:
#Cambiar el dato de los dos registros por ''TRANSPORTE DE PASAJEROS'
df_hechos['Acusado']= df_hechos['Acusado'].replace({'PASAJEROS':'TRANSPORTE DE PASAJEROS'})


In [100]:
#Verificar que se haya realizado el cambio correctamente
print(df_hechos['Acusado'].head(10))

0                       AUTO
1    TRANSPORTE DE PASAJEROS
2                       AUTO
3                         SD
4    TRANSPORTE DE PASAJEROS
5                OBJETO FIJO
6                       AUTO
7                       AUTO
8                     CARGAS
9                       AUTO
Name: Acusado, dtype: object


Variables geográficas Xy (caba)  , Pos x (indica la longitud), Pos y(indica la latitud)

In [101]:
df_hechos[df_hechos['Xy (caba)']== 'Point (. .)']

,Id,Cantidad víctimas,Fecha,Año,Mes,Día,Hora completa,Hora,Tipo de calle,Calle,Cruce,Lugar del hecho,Comuna,Xy (caba),Pos x,Pos y,Participantes,Víctima,Acusado
35,2016-0049,1,2016-04-17,2016,4,17,00:00:00,0,AUTOPISTA,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI,NO,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI,4,Point (. .),-58.37714647568196,-34.63657525428238,SD-SD,SD,SD
38,2016-0052,1,2016-04-20,2016,4,20,20:00:00,20,AUTOPISTA,"LUGONES, LEOPOLDO AV.",NO,AUTOPISTA LUGONES PK 10000,13,Point (. .),.,.,MOTO-SD,MOTO,SD
71,2016-0096,1,2016-07-25,2016,7,25,07:00:00,7,AUTOPISTA,AUTOPISTA DELLEPIANE LUIS TTE. GRAL.,NO,AUTOPISTA DELLEPIANE LUIS TTE. GRAL.,8,Point (. .),-58.47433193007387,-34.66684950051973,MOTO-CARGAS,MOTO,CARGAS
106,2016-0136,1,2016-10-25,2016,10,25,00:00:00,0,AUTOPISTA,AUTOPISTA BUENOS AIRES - LA PLATA,NO,AU BUENOS AIRES - LA PLATA KM. 4,4,Point (. .),.,.,MOTO-CARGAS,MOTO,CARGAS
119,2016-0151,1,2016-11-18,2016,11,18,20:35:00,20,CALLE,SD,NO,SD,0,Point (. .),.,.,PEATON-SD,PEATON,SD
139,2016-0174,1,2016-12-27,2016,12,27,00:00:00,0,AUTOPISTA,AUTOPISTA 25 DE MAYO,NO,AUTOPISTA 25 DE MAYO,0,Point (. .),.,.,SD-SD,SD,SD
176,2017-0042,1,2017-04-10,2017,4,10,09:00:00,9,GRAL PAZ,"LUGONES, LEOPOLDO AV.",NO,"LUGONES, LEOPOLDO AV.",14,Point (. .),.,.,MOTO-CARGAS,MOTO,CARGAS
180,2017-0050,2,2017-04-28,2017,4,28,11:08:08,11,AUTOPISTA,AUTOPISTA PERITO MORENO,NO,AU PERITO MORENO Y RAMAL ENLACE AU1/AU6,9,Point (. .),.,.,MOTO-CARGAS,MOTO,CARGAS
181,2017-0051,1,2017-05-01,2017,5,1,03:47:47,3,AUTOPISTA,AUTOPISTA DELLEPIANE LUIS TTE. GRAL.,NO,AU DELLEPIANE 2400,7,Point (. .),.,.,AUTO-AUTO,AUTO,AUTO
256,2017-0140,1,2017-11-19,2017,11,19,23:22:17,23,AUTOPISTA,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI,NO,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI,4,Point (. .),.,.,MOTO-PASAJEROS,MOTO,TRANSPORTE DE PASAJEROS


No se encontró la información para completar estos datos.Se decide, por lo tanto, asignarles el valor de 0 (cero), para poder realizar el análisis correctamente.

In [102]:
df_hechos['Pos x'] = df_hechos['Pos x'].replace('.', 0)
df_hechos['Pos y'] = df_hechos['Pos y'].replace('.', 0)
df_hechos['Xy (caba)'] = df_hechos['Xy (caba)'].replace('Point (. .)', 0)

In [103]:
#Verificar los cambios, visualizando los registros 180 y 181 de las columnas 'Pos x', 'Pos y' y 'Xy (caba)'
print(df_hechos.loc[[180, 181], ['Pos x', 'Pos y', 'Xy (caba)']])

    Pos x Pos y Xy (caba)
180     0     0         0
181     0     0         0


Hacemos los chequeos finales antes de exportar el archivo

In [104]:
print(df_hechos.isnull().sum())

Id                   0
Cantidad víctimas    0
Fecha                0
Año                  0
Mes                  0
Día                  0
Hora completa        0
Hora                 0
Tipo de calle        0
Calle                0
Cruce                0
Lugar del hecho      0
Comuna               0
Xy (caba)            0
Pos x                0
Pos y                0
Participantes        0
Víctima              0
Acusado              0
dtype: int64


In [105]:
func.verificar_tipo_variable(df_hechos)

,nombre_campo,tipo_datos
0,Id,[<class 'str'>]
1,Cantidad víctimas,[<class 'int'>]
2,Fecha,[<class 'pandas._libs.tslibs.timestamps.Timest...
3,Año,[<class 'int'>]
4,Mes,[<class 'int'>]
5,Día,[<class 'int'>]
6,Hora completa,[<class 'datetime.time'>]
7,Hora,[<class 'int'>]
8,Tipo de calle,[<class 'str'>]
9,Calle,[<class 'str'>]


In [112]:
# Convertir las columnas específicas de tipo int a tipo str
df_hechos['Xy (caba)'] = df_hechos['Xy (caba)'].astype(str)
df_hechos['Pos x'] = df_hechos['Pos x'].astype(str)
df_hechos['Pos y'] = df_hechos['Pos y'].astype(str)

In [113]:
func.verificar_tipo_variable(df_hechos)

,nombre_campo,tipo_datos
0,Id,[<class 'str'>]
1,Cantidad víctimas,[<class 'int'>]
2,Fecha,[<class 'pandas._libs.tslibs.timestamps.Timest...
3,Año,[<class 'int'>]
4,Mes,[<class 'int'>]
5,Día,[<class 'int'>]
6,Hora completa,[<class 'datetime.time'>]
7,Hora,[<class 'int'>]
8,Tipo de calle,[<class 'str'>]
9,Calle,[<class 'str'>]


Exportamos

In [114]:
# Especificar la ruta y el nombre del archivo de Excel de destino
ruta_destino = 'Data_transformed\df_hechos_transformed.xlsx'
# Exportar el DataFrame a un archivo Excel con los cambios realizados
df_hechos.to_excel(ruta_destino, index=False)
print(f'Se ha exportado correctamente el DataFrame a {ruta_destino}')

Se ha exportado correctamente el DataFrame a Data_transformed\df_hechos_transformed.xlsx


Traemos la pestaña Víctimas del Excel de Homicidios
Se extraen los datos de la pestaña Víctimas que se encuentra en el Excel de homicidios y se observan las primeras filas con .head()

In [33]:
df_victimas= pd.read_excel('Data_raw\homicidios.xlsx', sheet_name='VICTIMAS')

df_victimas.head(3)

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,2016-01-01,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01 00:00:00
1,2016-0002,2016-01-02,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02 00:00:00
2,2016-0003,2016-01-03,2016,1,3,CONDUCTOR,MOTO,MASCULINO,30,2016-01-03 00:00:00


Muestra de datos generales:

In [34]:
#Cantidad de columnas
df_victimas.columns

Index(['ID_hecho', 'FECHA', 'AAAA', 'MM', 'DD', 'ROL', 'VICTIMA', 'SEXO',
       'EDAD', 'FECHA_FALLECIMIENTO'],
      dtype='object')

In [35]:
#Cantidad de registros
print('Cantidad de registros:', df_victimas.shape[0])

Cantidad de registros: 717


Se estandarizan los nombres de las columnas

In [36]:
# Se coloca la primera en mayúscula
df_victimas.columns = [x.capitalize() for x in df_victimas.columns]
# Se reemplazan los guiones por espacios
df_victimas.columns = df_victimas.columns.str.replace('_', ' ')
# Se renombran algunas columnas
df_victimas = df_victimas.rename(columns={  'Aaaa':'Año',
                                            'Mm':'Mes',
                                            'Dd':'Día',
                                            'Victima':'Víctima'})

df_victimas.columns

Index(['Id hecho', 'Fecha', 'Año', 'Mes', 'Día', 'Rol', 'Víctima', 'Sexo',
       'Edad', 'Fecha fallecimiento'],
      dtype='object')

Datos nulos

In [37]:
print(df_victimas.isnull().sum())

Id hecho               0
Fecha                  0
Año                    0
Mes                    0
Día                    0
Rol                    0
Víctima                0
Sexo                   0
Edad                   0
Fecha fallecimiento    0
dtype: int64


Datos duplicados

In [38]:
df_victimas[df_victimas.duplicated()]

,Id hecho,Fecha,Año,Mes,Día,Rol,Víctima,Sexo,Edad,Fecha fallecimiento


Verificamos si el Id es único o hay duplicados.

In [40]:
def verifica_duplicados_por_columna(df, columna):
    # Se filtran las filas duplicadas
    duplicated_rows = df[df.duplicated(subset=columna, keep=False)]
    if duplicated_rows.empty:
        return "No hay duplicados"
    
    # se ordenan las filas duplicadas para comparar entre sí
    duplicated_rows_sorted = duplicated_rows.sort_values(by=columna)
    return duplicated_rows_sorted

# Llamar a la función para verificar duplicados en la columna 'Id'
duplicados_id_hecho = verifica_duplicados_por_columna(df_victimas, 'Id hecho')
print(duplicados_id_hecho)

      Id hecho      Fecha   Año  Mes  Día                   Rol Víctima  \
29   2016-0041 2016-03-29  2016    3   29             CONDUCTOR    MOTO   
30   2016-0041 2016-03-29  2016    3   29  PASAJERO_ACOMPAÑANTE    MOTO   
98   2016-0126 2016-09-18  2016    9   18             CONDUCTOR    AUTO   
99   2016-0126 2016-09-18  2016    9   18  PASAJERO_ACOMPAÑANTE    AUTO   
163  2017-0026 2017-02-26  2017    2   26  PASAJERO_ACOMPAÑANTE    AUTO   
164  2017-0026 2017-02-26  2017    2   26             CONDUCTOR    AUTO   
173  2017-0035 2017-03-23  2017    3   23             CONDUCTOR    AUTO   
174  2017-0035 2017-03-23  2017    3   23  PASAJERO_ACOMPAÑANTE    AUTO   
175  2017-0035 2017-03-23  2017    3   23  PASAJERO_ACOMPAÑANTE    AUTO   
177  2017-0036 2017-03-29  2017    3   29             CONDUCTOR    MOTO   
176  2017-0036 2017-03-29  2017    3   29  PASAJERO_ACOMPAÑANTE    MOTO   
186  2017-0050 2017-04-28  2017    4   28             CONDUCTOR    MOTO   
187  2017-0050 2017-04-28

Tipos de variable por columna

In [41]:
#Vemos el tipo de dato en cada columna.
def verificar_tipo_variable(df):
    mi_dict = {"nombre_campo": [], "tipo_datos": []}

    for columna in df.columns:
        mi_dict["nombre_campo"].append(columna)
        mi_dict["tipo_datos"].append(df[columna].apply(type).unique())
    df_info = pd.DataFrame(mi_dict)
        
    return df_info

# Llamar a la función para verificar el tipo de dato
tipo_dato = verificar_tipo_variable(df_victimas)
print(tipo_dato)


          nombre_campo                                         tipo_datos
0             Id hecho                                    [<class 'str'>]
1                Fecha  [<class 'pandas._libs.tslibs.timestamps.Timest...
2                  Año                                    [<class 'int'>]
3                  Mes                                    [<class 'int'>]
4                  Día                                    [<class 'int'>]
5                  Rol                                    [<class 'str'>]
6              Víctima                                    [<class 'str'>]
7                 Sexo                                    [<class 'str'>]
8                 Edad                     [<class 'int'>, <class 'str'>]
9  Fecha fallecimiento       [<class 'datetime.datetime'>, <class 'str'>]


'Edad' y 'Fecha fallecimiento'

In [42]:
# Cantidad de valores por tipo de dato en la columna 'Edad'
df_victimas['Edad'].apply(type).value_counts()

Edad
<class 'int'>    664
<class 'str'>     53
Name: count, dtype: int64

In [43]:
# Se observan los registros con tipo de dato str
print('Registro con tipo str:')
df_victimas[df_victimas['Edad'].apply(lambda x: isinstance(x, str))]

Registro con tipo str:


,Id hecho,Fecha,Año,Mes,Día,Rol,Víctima,Sexo,Edad,Fecha fallecimiento
30,2016-0041,2016-03-29,2016,3,29,PASAJERO_ACOMPAÑANTE,MOTO,MASCULINO,SD,2016-03-30 00:00:00
33,2016-0045,2016-04-11,2016,4,11,CONDUCTOR,MOTO,MASCULINO,SD,SD
35,2016-0048,2016-04-15,2016,4,15,PEATON,PEATON,FEMENINO,SD,SD
36,2016-0049,2016-04-17,2016,4,17,SD,SD,SD,SD,SD
39,2016-0052,2016-04-20,2016,4,20,SD,MOTO,SD,SD,SD
55,2016-0077,2016-06-13,2016,6,13,PEATON,PEATON,FEMENINO,SD,SD
63,2016-0085,2016-06-29,2016,6,29,SD,MOTO,MASCULINO,SD,SD
72,2016-0096,2016-07-25,2016,7,25,CONDUCTOR,MOTO,MASCULINO,SD,SD
89,2016-0115,2016-09-02,2016,9,2,SD,SD,MASCULINO,SD,SD
93,2016-0119,2016-09-04,2016,9,4,PASAJERO_ACOMPAÑANTE,SD,FEMENINO,SD,SD


se procede a buscar el valor Promedio en la columna Edad, pero teniendo en cuenta el Sexo de cada víctima.

También se observan valores faltantes en la columna 'Sexo', por lo cual para realizar la ingesta de datos en 'Edad', primero debemos completar la columna 'Sexo'.

In [44]:
def ingresa_valor_frecuente(df, columna):
    #Se reemplaza 'SD' con NaN en la columna especificada
    df[columna]=df[columna].replace('SD', pd.NA)
    
    #Verifica el valor más frecuente en la columna
    valor_frecuente= df[columna].mode().iloc[0]
    print('El valor más frecuente de', columna, 'es:', valor_frecuente)

    #Se modifican los valores NaN, con el valor más frecuente
    df[columna].fillna(valor_frecuente, inplace=True)



# Llamar a la función 
valor_frecuente_sexo = ingresa_valor_frecuente(df_victimas, 'Sexo')
print(valor_frecuente_sexo)


El valor más frecuente de Sexo es: MASCULINO
None


In [45]:
#Una vez modificada la columna 'Sexo', se modifica la columna 'Edad'
def ingresa_edad_media_segun_sexo(df):
    #Reemplaza 'SD' con NaN en la columna 'Edad'.
    df['Edad']= df['Edad'].replace('SD', pd.NA)

    #Se calcula el valor promedio de la edad, para cada género(es decir, agrupamos por género).
    promedio_por_genero= df.groupby('Sexo')['Edad'].mean()
    print(f'La edad promedio de Femenino es {round(promedio_por_genero["FEMENINO"])} y de Masculino es {round(promedio_por_genero["MASCULINO"])}')

    #Modificamos los valores NaN en 'Edad', con los valores correspondientes en cada género.
    #Se verifica fila por fila, si el valor en 'Edad' es NaN, se aplica la función, sino se devuelve el valor original.
    df['Edad']= df.apply(lambda row: promedio_por_genero[row['Sexo']] if pd.isna(row['Edad']) else row['Edad'], axis=1)

    #Convertimos el valor ingresado a tipo int
    df['Edad']= df['Edad'].astype(int)


# Llamar a la función 
edad_media_sexo = ingresa_edad_media_segun_sexo(df_victimas)
print(edad_media_sexo)

La edad promedio de Femenino es 51 y de Masculino es 40
None


In [47]:
#Revisamos que se hayan realizado los cambios correctamente en alguno de los ejemplos que veíamos antes.
df_victimas[df_victimas['Id hecho']=='2016-0174']

,Id hecho,Fecha,Año,Mes,Día,Rol,Víctima,Sexo,Edad,Fecha fallecimiento
141,2016-0174,2016-12-27,2016,12,27,SD,SD,MASCULINO,39,SD


en las columnas 'Rol' y 'Víctima', hay datos faltantes.

In [48]:
print('La cantidad de registros con SD en la columna Rol, es de:', len(df_victimas[df_victimas['Rol']=='SD']))
print('La cantidad de registros con SD en la columna Víctima, es de:', len(df_victimas[df_victimas['Víctima']=='SD']))

La cantidad de registros con SD en la columna Rol, es de: 11
La cantidad de registros con SD en la columna Víctima, es de: 9


In [49]:
#modificamos la columna Rol
func.ingresa_valor_frecuente(df_victimas, 'Rol')
#Modificamos la columna Víctima
func.ingresa_valor_frecuente(df_victimas, 'Víctima')

El valor más frecuente de Rol es: CONDUCTOR
El valor más frecuente de Víctima es: MOTO


In [51]:
#Volvemos a revisar, para verificar que se hayan realizado los cambios.
print('La cantidad de registros con SD en la columna Rol, es de:', len(df_victimas[df_victimas['Rol']=='SD']))
print('La cantidad de registros con SD en la columna Víctima, es de:', len(df_victimas[df_victimas['Víctima']=='SD']))
#Vemos el ejemplo que teníamos antes
df_victimas[df_victimas['Id hecho']=='2016-0174']

La cantidad de registros con SD en la columna Rol, es de: 0
La cantidad de registros con SD en la columna Víctima, es de: 0


,Id hecho,Fecha,Año,Mes,Día,Rol,Víctima,Sexo,Edad,Fecha fallecimiento
141,2016-0174,2016-12-27,2016,12,27,CONDUCTOR,MOTO,MASCULINO,39,SD


en la columna 'Fecha fallecimiento' se unifica el tipo de dato.

In [52]:
# Contar el número de registros 'SD' en la columna 'Fecha fallecimiento'
num_sd = df_victimas[df_victimas['Fecha fallecimiento'] == 'SD'].shape[0]

# Calcular el total de filas en el DataFrame
total_registros = df_victimas.shape[0]

# Calcular el porcentaje de registros 'SD'
porcentaje_sd = (num_sd / total_registros) * 100

print(f"El porcentaje de registros 'SD' en la columna 'Fecha fallecimiento' es: {porcentaje_sd:.2f}%")

El porcentaje de registros 'SD' en la columna 'Fecha fallecimiento' es: 9.48%


Para no perder los registros decidimos asignarles el valor de la fecha en la que ocurrio el accidente

In [53]:
# Copiar la columna 'Fecha' a la columna 'Fecha fallecimiento' donde el valor sea 'SD'
df_victimas.loc[df_victimas['Fecha fallecimiento'] == 'SD', 'Fecha fallecimiento'] = df_victimas['Fecha']

# Verificar que se hayan completado correctamente los registros
print(df_victimas[df_victimas['Fecha fallecimiento'] == 'SD'])

Empty DataFrame
Columns: [Id hecho, Fecha, Año, Mes, Día, Rol, Víctima, Sexo, Edad, Fecha fallecimiento]
Index: []


Eliminación de columnas
Eliminamos columnas cuyos datos puedeno obtenerse del df_hechos por estar vinculados

In [58]:
df_victimas=df_victimas.drop(['Fecha', 'Año', 'Mes', 'Día'], axis=1)
df_victimas.columns

Index(['Id hecho', 'Rol', 'Víctima', 'Sexo', 'Edad', 'Fecha fallecimiento'], dtype='object')

Verificamos una vez mas el tipo de variabes de las columnas

In [56]:
func.verificar_tipo_variable(df_victimas)

,nombre_campo,tipo_datos
0,Id hecho,[<class 'str'>]
1,Fecha,[<class 'pandas._libs.tslibs.timestamps.Timest...
2,Año,[<class 'int'>]
3,Mes,[<class 'int'>]
4,Día,[<class 'int'>]
5,Rol,[<class 'str'>]
6,Víctima,[<class 'str'>]
7,Sexo,[<class 'str'>]
8,Edad,[<class 'int'>]
9,Fecha fallecimiento,"[<class 'datetime.datetime'>, <class 'pandas._..."


In [57]:
print(df_victimas.isnull().sum())

Id hecho               0
Fecha                  0
Año                    0
Mes                    0
Día                    0
Rol                    0
Víctima                0
Sexo                   0
Edad                   0
Fecha fallecimiento    0
dtype: int64


Asignamos una clave unica para cada registro de victimas 'Id víctima'

In [63]:
# Reiniciar el índice para asignar una clave única a cada registro
df_victimas_con_clave = df_victimas.reset_index(drop=True)

# Verificar el DataFrame con la nueva clave única
print(df_victimas_con_clave)

# Asignar el nombre 'Id victima' a la columna de clave única
df_victimas_con_clave = df_victimas_con_clave.rename(columns={'index': 'Id víctima'})

# Verificar el DataFrame con la nueva columna 'Id victima'
print(df_victimas_con_clave)

      Id hecho                   Rol Víctima       Sexo  Edad  \
0    2016-0001             CONDUCTOR    MOTO  MASCULINO    19   
1    2016-0002             CONDUCTOR    AUTO  MASCULINO    70   
2    2016-0003             CONDUCTOR    MOTO  MASCULINO    30   
3    2016-0004             CONDUCTOR    MOTO  MASCULINO    18   
4    2016-0005             CONDUCTOR    MOTO  MASCULINO    29   
..         ...                   ...     ...        ...   ...   
712  2021-0092                PEATON  PEATON   FEMENINO    50   
713  2021-0093  PASAJERO_ACOMPAÑANTE    MOTO   FEMENINO    18   
714  2021-0094  PASAJERO_ACOMPAÑANTE    MOTO   FEMENINO    43   
715  2021-0095             CONDUCTOR    MOTO  MASCULINO    27   
716  2021-0096             CONDUCTOR    AUTO  MASCULINO    60   

     Fecha fallecimiento  Id víctima  
0    2016-01-01 00:00:00           1  
1    2016-01-02 00:00:00           2  
2    2016-01-03 00:00:00           3  
3    2016-01-10 00:00:00           4  
4    2016-02-01 00:00:00

Exportamos

In [64]:
# Especifica la ruta y el nombre del archivo de Excel de destino
ruta_destino = 'Data_transformed\df_victimas_transformed.xlsx'
# Exportar el DataFrame a un archivo Excel con los cambios realizados
df_victimas.to_excel(ruta_destino, index=False)
print(f'Se ha exportado correctamente el DataFrame a {ruta_destino}')

Se ha exportado correctamente el DataFrame a Data_transformed\df_victimas_transformed.xlsx
